# 01 - BigQuery - Table Data Source

---
## Source Data

**Overview**

This notebook imports source data for this project into Google BigQuery.  All the remaining notebooks utilize BigQuery as the source and leverage API's native to the machine learning approaches they feature.

In the enviornment setup notebook (00), a BigQuery source table was exported to CSV format in a Cloud Storage Bucket. This notebook, `01 - BigQuery - Table Data Source`, starts the machine learning lifecycle by importing source data and preparing it for machine learning.

All of these workflows utilize tabular data to fit a supervised learning model: predict a target variable by learning patterns in feature columns.  The type of supervised learning used in these projects is classification: models with a target variable that has multiple discrete classes.  

**The Data**

The source data is first exported to Google Cloud Storage in CSV format below.  The BigQuery source table is `bigquery-public-data.ml_datasets.ulb_fraud_detection`.  This is a table of credit card transactions that are classified as fradulant, `Class = 1`, or normal `Class = 0`.    
- The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).
- Read mode about BigQuery public datasets [here](https://cloud.google.com/bigquery/public-data)

**Description of the Data**

This is a table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.  In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction

**Preparation of the Data**

This notebook adds two columns to the source data and stores it in a new table with suffix `_prepped`.  
- `transaction_id` (string) a unique id for the row/transaction
- `splits` (string) this divided the tranactions into sets for `TRAIN` (80%), `VALIDATE` (10%), and `TEST` (10%)

---
## Setup

In [69]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'elite-monolith-466520-q3'

In [70]:
REGION = 'europe-west3'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

packages:

In [71]:
from google.cloud import bigquery
from google.cloud import storage

clients:

In [72]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

parameters:

In [73]:
BUCKET = PROJECT_ID

---
## Store the Source Data in GCS Storage Bucket
Check to see if the export exist and create if not:
- export from bigquery table to GCS bucket as CSV


In [74]:
file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"

In [75]:
# Create bucket if not exists

bucket = gcs.bucket(BUCKET)
if not bucket.exists():
    bucket = gcs.create_bucket(BUCKET, location=REGION)
    print(f"Bucket created: {bucket.name}")
else:
    print(f"Bucket exists: {bucket.name}")

Bucket created: elite-monolith-466520-q3


In [76]:
# export table into the bucket

bucketDef = gcs.bucket(BUCKET)
print("bucketDef name:",bucketDef)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Creating the export ...')
    extract.result()
    print(f'Exported the table to: gs://{bucketDef.name}/{file}')

bucketDef name: <Bucket: elite-monolith-466520-q3>
Creating the export ...
Exported the table to: gs://elite-monolith-466520-q3/01/01/data/fraud.csv


list files in the bucket:

In [94]:
print(list(bucketDef.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}')))
print(f'Review the files in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')


[<Blob: elite-monolith-466520-q3, 01/01/data/fraud.csv, 1753639418210223>]
Review the files in the console here:
https://console.cloud.google.com/storage/browser/elite-monolith-466520-q3/01;tab=objects&project=elite-monolith-466520-q3


---
## Create BigQuery Dataset

List BigQuery datasets in the project:

In [95]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud
retailrocket


In [80]:
# Create the dataset if missing:

ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

---
## Create BigQuery Table
- import data from Cloud Storage Bucket

In [82]:
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

Creating Table ...
Finished creating table: elite-monolith-466520-q3.fraud.fraud


### Retrieve and Review a Sample From The Table:
> **Note:** The `LIMIT 5` statement does limit the number of rows returned by BigQuery to 5 but BigQuery still does a full table scan.  If you have a table larger than 1GB and want to limit the rows scanned for a quick review like then then replacing `LIMIT 5` with `TABLESAMPLE SYSTEM (1 PERCENT)` would be more efficient.

In [83]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,150336,2.030851,-0.235559,-2.804940,-0.547376,2.441730,3.277629,-0.652959,0.843118,0.542395,...,-0.058377,-0.092950,0.203618,0.642094,-0.105975,0.502090,-0.024706,-0.040995,0.78,0
1,84068,-0.679466,1.334849,1.367987,0.856085,0.000189,-0.701735,0.582134,-0.003885,-0.741823,...,0.077499,0.311384,-0.095417,0.413387,-0.212580,-0.457638,-0.250283,-0.139162,0.78,0
2,122091,2.141596,0.077735,-2.104923,0.061232,0.368903,-1.825884,0.748172,-0.564815,0.354072,...,0.131269,0.490071,-0.035338,0.008649,0.415350,0.245709,-0.091809,-0.081979,0.78,0
3,136167,2.073311,0.265580,-1.745544,0.492756,0.286109,-1.427709,0.310907,-0.422277,0.505541,...,0.191552,0.767662,-0.043917,-0.156863,0.295408,-0.096150,0.002101,-0.031715,0.78,0
4,150523,2.032967,-0.387542,-0.505876,0.350922,-0.424833,-0.134032,-0.623411,-0.005357,1.481508,...,0.151034,0.737664,0.044700,-0.682962,-0.007459,-0.166488,0.047286,-0.044997,0.78,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142401,154599,0.667714,3.041502,-5.845112,5.967587,0.213863,-1.462923,-2.688761,0.677764,-3.447596,...,0.329760,-0.941383,-0.006075,-0.958925,0.239298,-0.067356,0.821048,0.426175,6.74,1
142402,90676,-2.405580,3.738235,-2.317843,1.367442,0.394001,1.919938,-3.106942,-10.764403,3.353525,...,10.005998,-2.454964,1.684957,0.118263,-1.531380,-0.695308,-0.152502,-0.138866,6.99,1
142403,34634,0.333499,1.699873,-2.596561,3.643945,-0.585068,-0.654659,-2.275789,0.675229,-2.042416,...,0.469212,-0.144363,-0.317981,-0.769644,0.807855,0.228164,0.551002,0.305473,18.96,1
142404,96135,-1.952933,3.541385,-1.310561,5.955664,-1.003993,0.983049,-4.587235,-4.892184,-2.516752,...,-1.998091,1.133706,-0.041461,-0.215379,-0.865599,0.212545,0.532897,0.357892,18.96,1


### Check out this table in BigQuery Console:
- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

In [84]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=elite-monolith-466520-q3


---
## Review Data in BigQuery

In [85]:
query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [86]:
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: Int64

In [87]:
df['Class'].value_counts(normalize=True)

Class
0    0.998273
1    0.001727
Name: proportion, dtype: Float64

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [88]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [89]:
(job.ended-job.started).total_seconds()

4.787

In [90]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

70.632136 MB


Review the test/train split:

In [91]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,TEST,28443,9.986763
1,VALIDATE,28600,10.041888
2,TRAIN,227764,79.971349


Retrieve a subset of the data to a Pandas dataframe:

In [92]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [93]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,28357,1.118515,0.010470,1.218695,2.642562,-0.490970,1.090685,-0.772611,0.434624,0.351791,...,-0.217638,-0.320151,0.655686,0.240463,0.039504,0.008324,0.0,0,ebad9bd1-8a95-49d3-a850-981b37871031,TEST
1,145765,-0.769016,0.497769,-1.491102,-0.484560,1.974915,-1.546615,1.086798,-0.020257,-0.960554,...,-0.673254,-0.908480,1.080143,0.786867,-0.239764,-0.265091,0.0,0,2811a3ff-eee5-497b-a8dd-c9ba668458c4,TEST
2,155336,2.032692,-0.201201,-0.684375,1.052115,-0.056531,0.259766,-0.616223,0.132281,1.088699,...,0.098126,-0.011056,0.000146,-0.547615,0.046057,-0.033400,0.0,0,632383b9-fea5-4db5-9547-c0cbb16a7450,TEST
3,45978,-0.718115,-0.172195,1.900310,0.788804,0.758189,-0.981929,0.070817,-0.188241,0.068257,...,0.118327,0.424479,-0.590786,0.145018,-0.045245,-0.045105,0.0,0,80f3cbef-0955-4283-b62e-d67be136b35c,TEST
4,142309,-0.278832,1.037789,-0.742498,-0.870529,0.669925,-1.440947,1.078175,0.046982,-0.330587,...,-0.051528,-0.067947,-0.774851,0.275841,0.031968,0.160919,0.0,0,6c3f044a-cea2-4753-8001-e6fa614d5504,TEST
